## Deliverable 2. Create a Customer Travel Destinations Map.

In [34]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,62.10,69,94,2.37,overcast clouds
1,1,Labuan,MY,5.2767,115.2417,78.96,88,100,8.12,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,78.51,71,74,12.59,light rain
3,3,Shingu,JP,33.7333,135.9833,45.91,85,100,6.40,overcast clouds
4,4,Bhimunipatnam,IN,17.8833,83.4333,67.51,63,7,5.35,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,62.10,69,94,2.37,overcast clouds
4,4,Bhimunipatnam,IN,17.8833,83.4333,67.51,63,7,5.35,clear sky
5,5,Bredasdorp,ZA,-34.5322,20.0403,65.28,80,96,14.76,overcast clouds
7,7,Mega,ET,4.0500,38.3000,64.38,32,50,6.49,scattered clouds
13,13,Kaitangata,NZ,-46.2817,169.8464,60.93,64,8,0.20,clear sky
22,22,Cape Town,ZA,-33.9258,18.4232,68.34,83,8,7.00,clear sky
23,23,Cidreira,BR,-30.1811,-50.2056,69.21,93,100,1.88,overcast clouds
26,26,Gat,IL,31.6100,34.7642,56.39,81,11,2.48,few clouds
30,30,Hargeysa,SO,9.5600,44.0650,60.26,72,90,7.81,overcast clouds
31,31,Pangkalanbuun,ID,-2.6833,111.6167,74.26,96,100,1.72,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                202
City                   202
Country                200
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                200
City                   200
Country                200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,New Norfolk,AU,62.10,overcast clouds,-42.7826,147.0587,
4,Bhimunipatnam,IN,67.51,clear sky,17.8833,83.4333,
5,Bredasdorp,ZA,65.28,overcast clouds,-34.5322,20.0403,
7,Mega,ET,64.38,scattered clouds,4.0500,38.3000,
13,Kaitangata,NZ,60.93,clear sky,-46.2817,169.8464,
22,Cape Town,ZA,68.34,clear sky,-33.9258,18.4232,
23,Cidreira,BR,69.21,overcast clouds,-30.1811,-50.2056,
26,Gat,IL,56.39,few clouds,31.6100,34.7642,
30,Hargeysa,SO,60.26,overcast clouds,9.5600,44.0650,
31,Pangkalanbuun,ID,74.26,overcast clouds,-2.6833,111.6167,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             20
dtype: int64

In [38]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('',np.nan, inplace=True)
hotel_clean_df = hotel_df.dropna()
hotel_clean_df.count()

City                   180
Country                180
Max Temp               180
Current Description    180
Lat                    180
Lng                    180
Hotel Name             180
dtype: int64

In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))